# The Weisfeiler-Lehman Isomorphism Test

The Weisfeiler-Lehman Isomorphism Test, also called WL-Test, is a classical result from the graph theory. It is an heuristic to find out if two graphs are isomorphic. The problem of the graph isomorphism does not have a definitive solution in polinomial time yet, making some people say it may be NP-Complete. The WL-Test offers one alternative that allows us to estimate it.

However, since this is an heuristic, the test is not perfect. Therefore, it fail in some simple cases and because of that new versions of this test were proposed.

In this notebook we will explore a little about the theory of the test and implement it in its classical form and also some newer versions.

In [1]:
import networkx as nx
def load_graph(file):
    G = nx.read_graphml(file).to_undirected()
    G.graph['phrase'] = G.graph.get('phrase', 'No phrase found')
    return G

# New implementation (gpt)

In [45]:
from __future__ import annotations
from pathlib import Path
from collections import defaultdict
from collections import Counter
import copy

# ---------- WL over trees ----------
def wl_tree_signature(G: nx.Graph) -> str:
    """
    1-WL color refinement on an (unlabeled) tree G.
    Returns a canonical signature string usable as a dict key.
    """
    # Relabel to 0..n-1 for array-friendly processing
    G = nx.relabel.convert_node_labels_to_integers(G, ordering="sorted")  # keeps attrs by default
    n = G.number_of_nodes()

    # adjacency as list of lists for speed
    adj = [list(G.neighbors(v)) for v in range(n)]  # neighbors() yields iterator.  [oai_citation:6‡networkx.org](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html?utm_source=chatgpt.com)
    labels = nx.get_node_attributes(G, 'label')  # node labels
    colors = [labels[u] for u in G.nodes()]  # node labels as colors
    tmp = [None] * n
    i = 0
    signature = ""
    while True:      
        
        for v in range(n):
            neigh_cols = sorted(colors[u] for u in adj[v])
            tmp[v] = str(colors[v]) +"_"+ str(tuple(neigh_cols))
        # compress tuples → small ints
        mapping = {}
        next_c = 0
        new_colors = [0] * n
        for v in range(n):
            key = tmp[v]
            #print(f"key is {key}")
            if key not in mapping:
                mapping[key] = next_c
                next_c += 1
            new_colors[v] = mapping[key]
        if new_colors == colors:
            break
        colors = new_colors
    hist = defaultdict(int)
    
    for c in tmp:
        hist[c] += 1
    signature += "|".join(f"{c}:{hist[c]}" for c in sorted(hist))
    return signature


In [46]:
from pathlib import Path
GRAPH_DIR = "../data/graphs_labeled_filtered/UD_Spanish-GSD"
# --------- CONFIG ---------
FOLDER = Path(GRAPH_DIR)
OUT_CSV = Path("isomorphic_groups_labeled_filtered.csv")

In [47]:
from tqdm import tqdm  # progress bar for long operations

groups = defaultdict(list)

for path in tqdm(FOLDER.rglob("*.graphml")):
    try:
         G = load_graph(path)
    except Exception as e:
        print(f"[WARN] Could not read {path}: {e}")
        continue

    sig = wl_tree_signature(G)

    groups[sig].append(str(path))

# --------- Write result ---------
with OUT_CSV.open("w", encoding="utf-8") as f:
    f.write("signature;count;files\n")
    for sig, files in groups.items():
        f.write(f"{sig};{len(files)};\"{'|'.join(files)}\"\n")

print(f"Done. Wrote {len(groups)} isomorphism classes to {OUT_CSV}")

10842it [00:06, 1743.61it/s]

Done. Wrote 10242 isomorphism classes to isomorphic_groups_labeled_filtered.csv


In [48]:
import pandas as pd

df = pd.read_csv("isomorphic_groups_labeled_filtered.csv", sep=";")
df


,signature,count,files
0,"0_(3,):1|1_(3,):1|2_(3,):1|3_(0, 1, 2, 6, 8):1...",2,../data/graphs_labeled_filtered/UD_Spanish-GSD...
1,"0_(11,):1|10_(11,):1|11_(0, 10, 13):1|12_(13,)...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
2,"0_(11,):1|10_(12,):1|11_(0, 1, 32, 36):1|12_(9...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
3,"0_(21,):1|10_(21,):1|11_(13,):1|12_(13,):1|13_...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
4,"0_(4,):1|10_(2, 3, 4, 9):1|11_(2,):1|1_(2,):1|...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
...,...,...,...
10237,"0_(6,):1|10_(9, 11, 14):1|11_(10,):1|12_(14,):...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
10238,"0_(22,):1|10_(9,):1|11_(22,):1|12_(3, 6, 14, 3...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
10239,"0_(5,):1|10_(2, 7, 8):1|11_(2,):1|1_(2,):1|2_(...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...
10240,"0_(9,):1|10_(11,):1|11_(9, 10, 13):1|12_(13,):...",1,../data/graphs_labeled_filtered/UD_Spanish-GSD...


In [49]:
df["largo_signature"] = df["signature"].apply(lambda x: len(x))

In [52]:
df[df["count"]>5].sort_values(["largo_signature","count"], ascending=False)["signature"].values[0]

'0_(30,):1|10_(8, 8, 12):1|11_(12,):1|12_(2, 7, 10, 11):1|13_(18,):1|14_(16,):1|15_(16,):1|16_(14, 15, 18):1|17_(18,):1|18_(2, 13, 16, 17):1|19_(24,):1|1_(2,):1|20_(22,):1|21_(22,):1|22_(20, 21, 24):1|23_(24,):1|24_(2, 19, 22, 23):1|25_(31,):1|26_(28,):1|27_(28,):1|28_(26, 27, 31):1|29_(31,):1|2_(1, 6, 12, 18, 24, 30, 31, 39, 47, 54, 55):1|30_(0, 2, 9, 9, 41):1|31_(2, 25, 28, 29, 34):1|32_(34,):1|33_(34,):1|34_(31, 32, 33):1|35_(39,):1|36_(38,):1|37_(38,):1|38_(36, 37, 39):1|39_(2, 35, 38, 42):1|3_(6,):1|40_(42,):2|41_(30,):1|42_(39, 40, 40):1|43_(47,):1|44_(46,):1|45_(46,):1|46_(44, 45, 47):1|47_(2, 43, 46, 53):1|48_(53,):1|49_(52, 53):1|4_(6,):1|50_(52,):1|51_(55,):1|52_(49, 50):1|53_(47, 48, 49):1|54_(2,):1|55_(2, 51, 57):1|56_(57,):1|57_(55, 56):1|5_(6,):2|6_(2, 3, 4, 5, 5):1|7_(12,):1|8_(10,):2|9_(30,):2'

In [53]:
df[df["signature"] == '0_(30,):1|10_(8, 8, 12):1|11_(12,):1|12_(2, 7, 10, 11):1|13_(18,):1|14_(16,):1|15_(16,):1|16_(14, 15, 18):1|17_(18,):1|18_(2, 13, 16, 17):1|19_(24,):1|1_(2,):1|20_(22,):1|21_(22,):1|22_(20, 21, 24):1|23_(24,):1|24_(2, 19, 22, 23):1|25_(31,):1|26_(28,):1|27_(28,):1|28_(26, 27, 31):1|29_(31,):1|2_(1, 6, 12, 18, 24, 30, 31, 39, 47, 54, 55):1|30_(0, 2, 9, 9, 41):1|31_(2, 25, 28, 29, 34):1|32_(34,):1|33_(34,):1|34_(31, 32, 33):1|35_(39,):1|36_(38,):1|37_(38,):1|38_(36, 37, 39):1|39_(2, 35, 38, 42):1|3_(6,):1|40_(42,):2|41_(30,):1|42_(39, 40, 40):1|43_(47,):1|44_(46,):1|45_(46,):1|46_(44, 45, 47):1|47_(2, 43, 46, 53):1|48_(53,):1|49_(52, 53):1|4_(6,):1|50_(52,):1|51_(55,):1|52_(49, 50):1|53_(47, 48, 49):1|54_(2,):1|55_(2, 51, 57):1|56_(57,):1|57_(55, 56):1|5_(6,):2|6_(2, 3, 4, 5, 5):1|7_(12,):1|8_(10,):2|9_(30,):2'
]["files"].values[0].split("|")

['../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_7962.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_11394.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_1627.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_10982.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_5719.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_12544.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_8323.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_3129.graphml',
 '../data/graphs_labeled_filtered/UD_Spanish-GSD/es_gsd-ud-train_6484.graphml']